In [1]:
import os
import numpy as np
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from skimage.io import imread

In [2]:
def compute_metrics(mask1, mask2, num_classes=3):
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()

    # Overall pixel accuracy
    accuracy = np.mean(mask1_flat == mask2_flat)

    # Cohen’s Kappa
    kappa = cohen_kappa_score(mask1_flat, mask2_flat, labels=list(range(num_classes + 1)))

    # Confusion matrix
    cm = confusion_matrix(mask1_flat, mask2_flat, labels=list(range(num_classes + 1)))

    # Mean IoU
    intersection = np.diag(cm)
    union = np.sum(cm, axis=0) + np.sum(cm, axis=1) - intersection
    mean_iou = np.mean(intersection / (union + 1e-6))

    return accuracy, kappa, mean_iou

def compare_mask_folders(folder1, folder2, num_classes=3):
    files = sorted(os.listdir(folder1))

    total_accuracy, total_kappa, total_iou = 0, 0, 0
    n = 0

    for fname in files:
        path1 = os.path.join(folder1, fname)
        path2 = os.path.join(folder2, fname)
        if not os.path.exists(path2):
            print(f"Skipping {fname}, not in both folders.")
            continue

        mask1 = imread(path1)
        mask2 = imread(path2)
        if mask1.shape != mask2.shape:
            print(f"Skipping {fname}, shape mismatch.")
            continue

        acc, kappa, iou = compute_metrics(mask1, mask2, num_classes)
        total_accuracy += acc
        total_kappa += kappa
        total_iou += iou
        n += 1

    print(f"\nCompared {n} masks:")
    print(f"Average Pixel Accuracy: {total_accuracy / n:.4f}")
    print(f"Average Cohen’s Kappa:  {total_kappa / n:.4f}")
    print(f"Average Mean IoU:       {total_iou / n:.4f}")

compare_mask_folders("masks-elyse", "masks-neeharika", num_classes=3)


Compared 186 masks:
Average Pixel Accuracy: 0.9973
Average Cohen’s Kappa:  0.7329
Average Mean IoU:       0.4128
